# C++



Se utiliza el patron de diseño Composite para representar el arbol de procesos.

In [ ]:
%%writefile process.cpp
#include <sys/wait.h>
#include <unistd.h>
#include <iostream>
#include <list>

#define TIME_SLEEP 30
#define PROCESS_A "A"
#define PROCESS_B "B"
#define PROCESS_C "C"
#define PROCESS_D "D"
#define PROCESS_E "E"
#define PROCESS_F "F"
#define PROCESS_G "G"

class ProcessBase 
{
 public:
  ProcessBase(){};
  virtual void Execute() = 0;
  void PrintMessage(std::string name) 
  {
    std::cout << "Proceso " + name + " ( PID: " + std::to_string(getpid()) +
                     " | PPID: " + std::to_string(getppid()) + " )"
              << std::endl;
  }
  void PrintErrorMessage(std::string name)
  {
    std:: cout << "Error al crear el proceso " + name + "." << std::endl;
  }
};

class ProcessLead : public ProcessBase 
{
 private:
  std::string name_;

 public:
  ProcessLead(std::string name) { this->name_ = name; }
  void Execute() override 
  {
    this->PrintMessage(this->name_);
    sleep(TIME_SLEEP);
    exit(EXIT_SUCCESS);
  }
};

class ProcessParent : public ProcessBase 
{
 private:
  std::string name_;
  std::list<ProcessBase*> childs_;

 public:
  ProcessParent(std::string name, std::list<ProcessBase*> childs) 
  {
    this->name_ = name;
    this->childs_ = childs;
  }

  void Execute() 
  {
    this->PrintMessage(this->name_);
    int pid;
    for (ProcessBase* child : this->childs_) 
    {
      pid = fork();
      if (pid == 0) 
      {
        child->Execute();
      }
      if (pid < 0)
      {
        child->PrintErrorMessage(this->name_);
      } 
    }
    sleep(TIME_SLEEP);
    for (ProcessBase* child : this->childs_) 
    {
      wait(NULL);
    }
    exit(EXIT_SUCCESS);
  }
};

int main() 
{
  ProcessBase* pid_g = new ProcessLead(PROCESS_G);
  ProcessBase* pid_f = new ProcessLead(PROCESS_F);
  ProcessBase* pid_e = new ProcessLead(PROCESS_E);
  ProcessBase* pid_c = new ProcessLead(PROCESS_C);

  std::list<ProcessBase*> childs_b = {pid_e, pid_f};
  ProcessBase* pid_b = new ProcessParent(PROCESS_B, childs_b);

  std::list<ProcessBase*> childs_d = {pid_g};
  ProcessBase* pid_d = new ProcessParent(PROCESS_D, childs_d);

  std::list<ProcessBase*> childs_a = {pid_b, pid_c, pid_d};
  ProcessBase* pid_a = new ProcessParent(PROCESS_A, childs_a);

  pid_a->Execute();

  return EXIT_SUCCESS;
}

Writing process.cpp


Verificamos que existe el archivo Fork.cpp

In [ ]:
!ls -l

total 8
-rw-r--r-- 1 root root 2237 Apr 16 23:34 process.cpp
drwxr-xr-x 1 root root 4096 Apr 13 13:30 sample_data


Compilamos el archivo Fork.cpp y generamos el binario.

In [ ]:
%%shell
g++ process.cpp -o process.bin

Ponemos en ejecucion el programa obtenido, y mostramos la salida del mismo en el archivo "salidaC"

In [ ]:
%%shell
nohup ./process.bin 1> salidaC 2> /dev/null & 

Visualizamos los procesos generados por este programa

In [ ]:
%%shell
ps -ef | grep process.bin

root         355       1  0 23:34 ?        00:00:00 ./process.bin
root         356     355  0 23:34 ?        00:00:00 ./process.bin
root         357     355  0 23:34 ?        00:00:00 ./process.bin
root         358     356  0 23:34 ?        00:00:00 ./process.bin
root         359     355  0 23:34 ?        00:00:00 ./process.bin
root         360     356  0 23:34 ?        00:00:00 ./process.bin
root         361     359  0 23:34 ?        00:00:00 ./process.bin
root         370     149  0 23:34 ?        00:00:00 /bin/bash -c ps -ef | grep process.bin 
root         372     370  0 23:34 ?        00:00:00 grep process.bin


Ahora visualizamos el arbol de procesos, ejecutando el siguiente comando:
***pstree -pc PID***

**PID**: PID del proceso A (Lo pueden encontrar dentro del archivo "salidaC")

In [ ]:
%%shell
pstree -pc 355

process.bin(355)─┬─process.bin(356)─┬─process.bin(358)
                 │                  └─process.bin(360)
                 ├─process.bin(357)
                 └─process.bin(359)───process.bin(361)


Visualizamos la salida de este programa

In [ ]:
!cat salidaC

Proceso A ( PID: 355 | PPID: 1 )
Proceso B ( PID: 356 | PPID: 355 )
Proceso D ( PID: 359 | PPID: 355 )
Proceso G ( PID: 361 | PPID: 359 )
Proceso E ( PID: 358 | PPID: 356 )
Proceso F ( PID: 360 | PPID: 356 )
Proceso C ( PID: 357 | PPID: 355 )



# Java




El programa recibe 2 argumentos, la cantidad de hijos que debe generar el proceso al ejecutarse, y el nivel de jerarquia que este tiene. Para cumplimentar el ejercicio tiene un corte cuando alcanzan el nivel 2.

In [ ]:
%%writefile ProcessTree.java
package processtree;

import java.io.IOException;

public class ProcessTree 
{
  public static void main(String[] args) throws IOException, InterruptedException 
  {
    ProcessHandle process = ProcessHandle.current();
    long pid = process.pid();
    long ppid = process.parent().get().pid();
    System.out.println("Soy el proceso: " + pid + ", Mi papa es: " + ppid);

    int level = Integer.parseInt(args[1]);
    if (level == 2) 
	{
      Thread.sleep(10000);
      return;
    }

    int childrenNum = Integer.parseInt(args[0]);

    Process childs[] = new Process[childrenNum];

    for (int i = 1; i <= childrenNum; i++) 
	{
      ProcessBuilder pb =
          new ProcessBuilder(
              "java", "ProcessTree.java", String.valueOf(childrenNum - i), String.valueOf(level + 1));
      pb.inheritIO();
      childs[i - 1] = pb.start();
    }
    Thread.sleep(10000);
    for (int i = 0; i < childs.length; i++) 
	{
      childs[i].waitFor();
    }
  }
}


Writing ProcessTree.java


Ponemos en ejecucion el programa obtenido, y mostramos la salida del mismo en el archivo "salidaJava"

In [ ]:
!nohup java ProcessTree.java 3 0 1>salidaJava 2>/dev/null &

Visualizamos los procesos que produce este programa.

In [ ]:
!ps -ef | grep ProcessTree.java

root        1771       1 99 23:40 ?        00:00:02 java ProcessTree.java 3 0
root        1799    1771 47 23:40 ?        00:00:00 java ProcessTree.java 2 1
root        1810    1771  0 23:40 ?        00:00:00 java ProcessTree.java 1 1
root        1815    1771  0 23:40 ?        00:00:00 java ProcessTree.java 0 1
root        1857     149  0 23:40 ?        00:00:00 /bin/bash -c ps -ef | grep ProcessTree.java
root        1859    1857  0 23:40 ?        00:00:00 grep ProcessTree.java


Vemos el arbol de procesos que genero este programa.


In [ ]:
!pstree -pT 1771

java(1771)─┬─java(1799)─┬─java(1882)
           │            └─java(1888)
           ├─java(1810)───java(1915)
           └─java(1815)


Mostramos la salida del programa

In [ ]:
!cat salidaJava

Soy el proceso: 1771, Mi papa es: 1
Soy el proceso: 1799, Mi papa es: 1771
Soy el proceso: 1815, Mi papa es: 1771
Soy el proceso: 1810, Mi papa es: 1771
Soy el proceso: 1888, Mi papa es: 1799
Soy el proceso: 1882, Mi papa es: 1799
Soy el proceso: 1915, Mi papa es: 1810



# Python

Se utiliza el patron de diseño Composite para representar el arbol de procesos.

In [ ]:
%%writefile process.py
import os
import time
from abc import ABC, abstractmethod
from typing import List


PROCESO_A = "A"
PROCESO_B = "B"
PROCESO_C = "C"
PROCESO_D = "D"
PROCESO_E = "E"
PROCESO_F = "F"
PROCESO_G = "G"


class ProcessBase(ABC):
    def __init__(self, name):
        super().__init__()
        self._name = name

    @abstractmethod
    def execute(self):
        pass

    def print_message(self):
        print("Proceso " + self._name + " (PID: " + str(os.getpid()) +
              " | PPID: " + str(os.getppid()) + " )")

    def print_error_message(self):
        print("Error al crear el proceso " + self._name)


class ProcessLeaf(ProcessBase):
    def __init__(self, name):
        super().__init__(name)

    def execute(self):
        self.print_message()
        time.sleep(30)
        os._exit(1)


class ProcessParent(ProcessBase):
    def __init__(self, name, childs):
        super().__init__(name)
        self._childs: List[ProcessBase] = childs

    def execute(self):
        self.print_message()
        for child in self._childs:
            pid = os.fork()
            if pid == 0:
                child.execute()
            if pid < 0:
                self.print_error_message()

        time.sleep(30)
        for _ in self._childs:
            os.wait()
        os._exit(1)


def main():
    process_g = ProcessLeaf(PROCESO_G)
    process_f = ProcessLeaf(PROCESO_F)
    process_e = ProcessLeaf(PROCESO_E)
    process_c = ProcessLeaf(PROCESO_C)

    process_b = ProcessParent(PROCESO_B, [process_e, process_f])
    process_d = ProcessParent(PROCESO_D, [process_g])
    process_a = ProcessParent(PROCESO_A, [process_b, process_c, process_d])

    process_a.execute()


if __name__ == "__main__":
    main()


Overwriting process.py


Ponemos en ejecucion el programa obtenido, y mostramos la salida del mismo en el archivo "salidaPython"

In [52]:
!nohup python process.py 1>salidaPython 2>/dev/null &

Visualizamos los procesos en ejecucion del programa "main.py"



In [53]:
!ps -ef | grep process.py

root       17100       1  1 00:41 ?        00:00:00 python3 process.py
root       17102   17100  0 00:41 ?        00:00:00 python3 process.py
root       17103   17100  0 00:41 ?        00:00:00 python3 process.py
root       17104   17100  0 00:41 ?        00:00:00 python3 process.py
root       17105   17102  0 00:41 ?        00:00:00 python3 process.py
root       17106   17102  0 00:41 ?        00:00:00 python3 process.py
root       17107   17104  0 00:41 ?        00:00:00 python3 process.py
root       17116     149  0 00:41 ?        00:00:00 /bin/bash -c ps -ef | grep process.py
root       17118   17116  0 00:41 ?        00:00:00 grep process.py


Visualizamos el arbol de procesos que genera este programa.

In [55]:
!pstree -pc 17100

python3(17100)─┬─python3(17102)─┬─python3(17105)
               │                └─python3(17106)
               ├─python3(17103)
               └─python3(17104)───python3(17107)


Visualizamos la salida por consola

In [56]:
!cat salidaPython

In [50]:
!python3 process.py

Proceso A (PID: 16907 | PPID: 149 )
Proceso B (PID: 16908 | PPID: 16907 )
Proceso C (PID: 16909 | PPID: 16907 )
Proceso F (PID: 16912 | PPID: 16908 )
Proceso E (PID: 16911 | PPID: 16908 )
Proceso D (PID: 16910 | PPID: 16907 )
Proceso G (PID: 16913 | PPID: 16910 )
